In [36]:
# Generate TER Table from specified dataset
import pandas as pd

In [37]:
df = pd.read_csv("../csv/sample.csv")

In [38]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupCell,DropoffCell,Unnamed: 21
0,1,2015-07-13 14:29:57,2015-07-13 15:19:06,2,17.80,-73.787956,40.641628,2,N,-73.969269,...,52.0,0.0,0.5,0.00,5.54,0.3,58.34,70:61,27:64,NaN
1,1,2015-07-29 08:47:12,2015-07-29 08:53:28,1,0.70,-73.969505,40.763618,1,N,-73.962135,...,5.5,0.0,0.5,1.25,0.00,0.3,7.55,26:63,27:61,NaN
2,2,2015-07-20 14:19:43,2015-07-20 14:40:02,5,2.90,-73.964363,40.767414,1,N,-73.989037,...,14.5,0.0,0.5,3.82,0.00,0.3,19.12,27:62,27:71,NaN
3,1,2015-07-15 23:39:36,2015-07-15 23:46:13,1,1.20,-73.989815,40.733536,1,N,-74.007965,...,6.5,0.5,0.5,0.00,0.00,0.3,7.80,27:70,23:72,NaN
4,2,2015-07-04 17:15:32,2015-07-04 17:17:53,3,0.56,-73.983704,40.735992,1,N,-73.992203,...,4.0,0.0,0.5,0.96,0.00,0.3,5.76,27:69,26:70,NaN


In [39]:
# Convert string type to datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [40]:
# Total earninigs = tip_amount + total_amount
df['total_earnings'] = df["tip_amount"] + df['total_amount']

In [41]:
# Calculate trip_duration in seconds
df["trip_duration"] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()

In [42]:
# Extract day of week and minute of day
df["day_of_week"] = df["tpep_pickup_datetime"].dt.weekday.astype(str)
df["minute_of_day"] = df["tpep_pickup_datetime"].dt.time

In [50]:
# Round down time to nearest minute
df['minute_of_day'] =  [i.replace(second=0) for i in df['minute_of_day']]

In [51]:
# Compute frequency and TER
df_ter = df.groupby(["PickupCell", "day_of_week", "minute_of_day"]) \
            .size() \
            .to_frame(name='frequency') \
            .join(df.groupby(["PickupCell", "day_of_week", "minute_of_day"]) \
                    .agg({"total_earnings":"mean", "trip_duration":"mean"}) \
                    .rename(columns={"trip_duration":"avg(td)", "total_earnings":"avg(te)"}) \
            ).reset_index()

In [52]:
df_ter['avg(ter)'] = df_ter['avg(te)']/df_ter['avg(td)']

In [53]:
df_ter.to_csv("../csv/df_score_table.csv", index=False)

In [55]:
df_ter['day_of_week'][0]

'6'